# Baseline - Conectividad por medio de WhastApp y Twilio

In [1]:
from twilio.twiml.messaging_response import MessagingResponse
from flask import Flask, request, Response
from twilio.rest import Client
from dotenv import load_dotenv
import requests
import json
import os
import re


load_dotenv(dotenv_path="chatbot.env")

account_sid = os.getenv('account_sid_2')
auth_token  = os.getenv('auth_token_2')
FROM_NUMBER  = "whatsapp:+14155238886"
client = Client(account_sid, auth_token )

Definimos un diccionario para conocer el estado de cada chat

In [2]:
# Global session storage (temporary — resets if the app restarts)
user_session = {}

Definimos una función para enviar mensajes sin formato a WhatsApp

In [3]:
def send_whatsapp_message(From, To, message):

    try:
        
        msg = client.messages.create(
            from_=From,
            to=To,
            body=message
        )
        print(f"Message sent: {msg.sid}")
        return msg.sid
    
    except Exception as e:
        
        print(f"Error sending message: {e}")
        return None

Probamos la función de envío.

In [4]:
def send_menu(From, To):
    
    menu_message = (
        "👋 *Bienvenido, será un gusto atenderte.*\n\n"
        "¿Qué te interesa llevar a cabo?\n\n"
        "🅰️ *Hacer un reporte de un predio infectado*\n"
        "🅱️ *Preguntar por información*"
        "\n\n"
        "Actualmente, solo estas 2 opciones tenemos disponibles."
    )

    try:
        msg = client.messages.create(
            from_=From,
            to=To,
            body=menu_message
        )
        print(f"✅ Menu sent successfully! SID: {msg.sid}")
        return msg.sid
    except Exception as e:
        print(f"❌ Error sending menu: {e}")
        return None

In [5]:
def report_message_flow( sender, state, req ):

	# Cargamos el mensaje
	message = request.form.get("Body", "").strip().lower() # Texto enviado por el usuario
	label   = request.form.get("Label")  				   # Label de la ubicacion enviada
	lat = request.form.get("Latitude") 					   # Si envian ubicacion, cargamos la latitud
	lon = request.form.get('Longitude') 				   # Si envian ubicacion, cargamos la longitud
	address = request.form.get('Address') 				   # Tomamos la dirección generada por WhatsApp
	photo_url = req.form.get('MediaUrl0')
	num_media = int(req.form.get('NumMedia', 0))
 
	response_text = ""
	next_state = state
 
	if state == "saludo":
		if (message == "a") or (message == "🅰️"):
			response_text = ("Gracias por tu proactividad. Te guiaré para reportar el predio. Primero, envía la **ubicación** del lote afectado.")
			next_state  = "ask-location"
   
	elif state == "ask-location":
		if lat and lon:
			user_session[sender]["data"]["lat"]  = lat
			user_session[sender]["data"]["lon"] = lon
			user_session[sender]["data"]["address"]   = address
			response_text = (
				"📍 Ubicación recibida *correctamente*.\n"
				"Ahora, por favor envía una **foto** del lote o planta afectada."
			)
			next_state = "ask_photo"
		else:
			response_text = ("Parece que la información que enviaste no es correcta. Por favor, usa la función de ubicación de WhatsApp.")

	elif state == "ask_photo":
		if num_media > 0:
			photo_url = req.form.get('MediaUrl0')
			user_session[sender]["data"]["photo_url"] = photo_url
			response_text = (
				"📸 Foto recibida correctamente.\n"
				"Por último, clasifica el **nivel de riesgo** que observas: Alto, Medio o Bajo."
			)
			next_state = "ask_risk"
		else:
			response_text = (
				"⚠️ No se detectó una foto. Por favor envía una imagen del lote afectado."
			)

	elif state == "ask_risk":
    
		if message in ["alta", "media", "baja", "alto", "medio", "bajo"]:
			user_session[sender]["data"]["user_risk_assesment"] = message.capitalize()
			data = user_session[sender]["data"]
			response_text = (
				f"✅ Gracias por tu reporte.\n"
				f"📍 Ubicación: ({data['lat']}, {data['lon']})\n"
				f"📸 Foto: Confirmada\n"
				f"🚨 Riesgo: {data['user_risk_assesment']}\n\n"
				"¿Esta información es correcta? Responde con *Sí* o *No*."
			)
			next_state = "confirmation"

		else:
			response_text = (
				"Por favor indica el nivel de riesgo como: Alta, Media o Baja."
			)

	elif state == "confirmation":
		msg = message.strip().lower()

		if re.search(r'^\s*s[ií]\s*$', msg):
			response_text = (
				"🌾 Tu reporte ya fue registrado. ¡Gracias por tu colaboración!"
			)
			print(user_session[sender])
		elif re.search(r'^\s*no\s*$', msg):
			response_text = "❌ Entendido. Tu reporte no se registrará."
		else:
			response_text = "Por el momento, solo podemos aceptar respuestas como 'Sí' o 'No'."
	else:
		response_text = ("Por el momento, solo podemos aceptar respuestas como 'Sí', 'No', etc. ")

	return response_text, next_state

In [ ]:
# reiniciamos user_session para hacer pruebas

user_session = {}

app = Flask(__name__)

@app.route("/reply_whatsapp", methods=['POST'])

def reply_whatsapp():

	sender  = request.form.get("From") # Guardamos quien nos escribio
	message = request.form.get("Body", "").strip().lower() # Tomamos el texto del mensaje para mostrarlo en la consola
	resp = MessagingResponse()
 
	print(f"Mensaje recibido: --> {message}")
 
	if sender not in user_session:
		user_session[sender] = {"state": "saludo",
								"data" : {
									"lat" : None, 
        							"lon" : None,
									"address" : None,
									"photo_url" : None,
									"user_risk_assesment": None,
									"price" : None
								}
              					}

		send_menu(FROM_NUMBER, sender)
		resp.message("Quedo a la espera de tu respuesta.")
		return Response(str(resp), mimetype="text/xml")

	
	response_text, next_state = report_message_flow(sender, user_session[sender]['state'], request)
	user_session[sender]["state"] = next_state
	resp.message(response_text)
	
	return Response(str(resp), mimetype='text/xml')


if __name__ == "__main__":
	app.run(port=3000)